# 1. Installing PyMongo

In [1]:
! pip install pymongo

Defaulting to user installation because normal site-packages is not writeable


# 2.Importing libreries 

In [5]:
import datetime                            # Imports datetime library
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import numpy as np

# 3. MongoDB connection

In [6]:
# uri (uniform resource identifier) defines the connection parameters
uri = 'mongodb://localhost:27017/'
# start client to connect to MongoDB server
client = MongoClient( uri )

In [7]:
client.list_database_names()

['Lab1', 'admin', 'config', 'local']

# 4. Scrapping server Arabic web

In this step, I enhance the scraping process by implementing pagination functionality. This enables the program to scrape a larger number of articles by automatically navigating through multiple pages, particularly utilizing a 'See More' button to access additional content on the Al Jazeera News website. By incorporating this feature, the scraping process becomes more comprehensive and can gather a broader range of articles for analysis or other purposes.

In [22]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from urllib.parse import urljoin


def scrape_links_by_id(url, class_name, seeMore_button_class, filter_domain=None, max_clicks=5):
    try:
        # Set up the Selenium WebDriver (make sure you have installed the appropriate browser driver)
        driver = webdriver.Chrome()  # Change to the appropriate WebDriver for your browser

        # Open the URL in the WebDriver
        driver.get(url)

        # Scroll to the bottom of the page multiple times to ensure the button is in view
        #for _ in range(5):
        #    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #    time.sleep(1)  # Wait for a moment after each scroll

        # Initialize counter for button clicks
        click_count = 0

        # Find and click the "See More" button multiple times
        while click_count < max_clicks:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            show_more_button = driver.find_element(By.CLASS_NAME, seeMore_button_class)
            #driver.execute_script("arguments[0].click();", show_more_button)
            show_more_button.click()

            print("Clicked 'See More' button successfully")
            click_count += 1
            time.sleep(5)  # Wait for some time after clicking the button

        # Wait for some time to let the page load after clicking the button
        time.sleep(5)  # Adjust the waiting time as needed

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Find all elements with the specified class
        elements = soup.find_all(id=class_name)

        # Extract links from the elements
        links = []
        for element in elements:
            # Find all links within the element
            for link in element.find_all('a', href=True):
                href = link['href']
                if filter_domain:
                    if href.startswith(filter_domain):
                        links.append(href)
                else:
                    links.append(href)

        return links
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        # Close the WebDriver
        driver.quit()

In [25]:
url = 'https://www.aljazeera.net/news/'
id = "news-feed-container"
seeMore_button_class = "show-more-button"
pages_A = 5

links = scrape_links_by_id(url , id, seeMore_button_class, '/news/', max_clicks=10)
links

Clicked 'See More' button successfully
Clicked 'See More' button successfully
Clicked 'See More' button successfully
Clicked 'See More' button successfully
Clicked 'See More' button successfully
Clicked 'See More' button successfully
Clicked 'See More' button successfully
Clicked 'See More' button successfully
Clicked 'See More' button successfully
Clicked 'See More' button successfully


['/news/2024/4/6/%d9%83%d9%85%d9%8a%d9%86-%d9%84%d9%84%d9%82%d8%b3%d8%a7%d9%85-%d9%8a%d8%b3%d8%aa%d9%87%d8%af%d9%81-3-%d8%af%d8%a8%d8%a7%d8%a8%d8%a7%d8%aa-%d8%a5%d8%b3%d8%b1%d8%a7%d8%a6%d9%8a%d9%84%d9%8a%d8%a9',
 '/news/2024/4/6/%d8%ad%d9%85%d8%a7%d8%b3-16',
 '/news/2024/4/6/%d9%84%d8%a8%d9%86%d8%a7%d9%86-69',
 '/news/2024/4/6/%d8%ac%d9%8a%d8%b4-%d8%a7%d9%84%d8%a7%d8%ad%d8%aa%d9%84%d8%a7%d9%84-%d9%8a%d8%b9%d9%84%d9%86-%d8%a7%d8%b3%d8%aa%d8%b9%d8%a7%d8%af%d8%a9-%d8%ac%d8%ab%d8%a9-%d8%a3%d8%b3%d9%8a%d8%b1-%d9%82%d8%aa%d9%84',
 '/news/2024/4/6/%d9%88%d8%a7%d8%b4%d9%86%d8%b7%d9%86-%d8%aa%d8%ad%d8%b0%d8%b1-%d9%88%d8%af%d9%88%d9%84-%d8%ba%d8%b1%d8%a8%d9%8a%d8%a9-%d8%aa%d8%b6%d8%ba%d8%b7-%d9%84%d9%88%d9%82%d9%81-%d8%a7%d9%84%d8%ad%d8%b1%d8%a8',
 '/news/2024/4/6/%d8%b9%d9%85%d9%84%d9%8a%d8%a7%d8%aa-%d8%ac%d8%af%d9%8a%d8%af%d8%a9-%d9%84%d9%84%d9%85%d9%82%d8%a7%d9%88%d9%85%d8%a9-%d9%88%d8%a7%d9%84%d8%a7%d8%ad%d8%aa%d9%84%d8%a7%d9%84-%d9%8a%d9%82%d8%b1',
 '/news/2024/4/6/%d8%a7%d9%84%d9%85%d9%83%

In [26]:
links = ["https://www.aljazeera.net" + link for link in links]
links

['https://www.aljazeera.net/news/2024/4/6/%d9%83%d9%85%d9%8a%d9%86-%d9%84%d9%84%d9%82%d8%b3%d8%a7%d9%85-%d9%8a%d8%b3%d8%aa%d9%87%d8%af%d9%81-3-%d8%af%d8%a8%d8%a7%d8%a8%d8%a7%d8%aa-%d8%a5%d8%b3%d8%b1%d8%a7%d8%a6%d9%8a%d9%84%d9%8a%d8%a9',
 'https://www.aljazeera.net/news/2024/4/6/%d8%ad%d9%85%d8%a7%d8%b3-16',
 'https://www.aljazeera.net/news/2024/4/6/%d9%84%d8%a8%d9%86%d8%a7%d9%86-69',
 'https://www.aljazeera.net/news/2024/4/6/%d8%ac%d9%8a%d8%b4-%d8%a7%d9%84%d8%a7%d8%ad%d8%aa%d9%84%d8%a7%d9%84-%d9%8a%d8%b9%d9%84%d9%86-%d8%a7%d8%b3%d8%aa%d8%b9%d8%a7%d8%af%d8%a9-%d8%ac%d8%ab%d8%a9-%d8%a3%d8%b3%d9%8a%d8%b1-%d9%82%d8%aa%d9%84',
 'https://www.aljazeera.net/news/2024/4/6/%d9%88%d8%a7%d8%b4%d9%86%d8%b7%d9%86-%d8%aa%d8%ad%d8%b0%d8%b1-%d9%88%d8%af%d9%88%d9%84-%d8%ba%d8%b1%d8%a8%d9%8a%d8%a9-%d8%aa%d8%b6%d8%ba%d8%b7-%d9%84%d9%88%d9%82%d9%81-%d8%a7%d9%84%d8%ad%d8%b1%d8%a8',
 'https://www.aljazeera.net/news/2024/4/6/%d8%b9%d9%85%d9%84%d9%8a%d8%a7%d8%aa-%d8%ac%d8%af%d9%8a%d8%af%d8%a9-%d9%84%d9%84%d9%8

In [27]:
np.unique(links).shape

(97,)

# 5. Save contenue of every article in MongoDB 

In this step we have gone scrap the content of article link and save the raw data in MongoDB

## 5.1 Create MongoDB collection 

In [28]:
# Create Lab1 Database
db = client["Lab1"]
# Create Articles collection 
article_collection = db["articles"]

## 5.2 Store article content in the collection

In [31]:
def scrape_article_content(URLs, id):
    try:
        contents = []
        for url in URLs:
            # Send an HTTP request to fetch the HTML content
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
            soup = BeautifulSoup(response.content, 'html.parser')
            # Find all elements with the specified id
            elements = soup.find_all(id=id)
            # Extract text content from elements
            article_content = [element.text for element in elements]
            # Create a dictionary with the URL as key and article content as value
            article_dict = {"content": article_content}
            contents.append(article_dict)
        return contents
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch URL: {url}. Error: {e}")
        return None


In [32]:
id_article_content = "main-content-area"
contents_articles = scrape_article_content(links, id_article_content)

In [33]:
contents_articles

[{'content': ['أخبارعمليات نوعية للقسام توقع 14 قتيلا من جنود الاحتلال بخان يونسplay videoplay videoمدة الفيديو 01 minutes 33 seconds 01:336/4/2024-|آخر تحديث: 6/4/202407:05 م (بتوقيت مكة المكرمة)كبدت كتائب عز الدين القسام\xa0الجناح العسكري لحركة المقاومة الإسلامية (حماس) قوات الاحتلال الإسرائيلي 14 قتيلا وعدة إصابات في عمليات نوعية نفذتها اليوم السبت في محور خان يونس بقطاع غزة.\nوقالت كتائب القسام في بيان إن مقاتليها قتلوا 6 جنود إسرائيليين، وأصابوا آخرين، في منطقة الزنة شرقي خان يونس، موضحة أنها استهدفت 4 دبابات ميركافا بقذائف "الياسين 105″، مشيرة إلى أنه فور تقدم قوات الإنقاذ إلى المكان ووصولها إلى وسط حقل ألغام أُعد مسبقا، استُهدفت بتفجير 3 عبوات مضادة للأفراد.\nوأوضح بيان القسام أن المواجهات لا تزال مستمرة في منطقة الزنة، شرقي خان يونس.\nكما أعلنت كتائب القسام قتل 5 جنود إسرائيليين من مسافة صفر وإصابة آخرين وتدمير ناقلة جند بمنطقة حي الأمل غرب خان يونس، فضلا عن استهداف دبابة إسرائيلية أخرى بقذيفة الياسين 105 وقوة راجلة بعبوة وإيقاعها بين قتيل وجريح في خان يونس.\nوأفاد مراسل الجزير

In [34]:
article_collection.insert_many(contents_articles)

InsertManyResult([ObjectId('66117a6283889bac712643ea'), ObjectId('66117a6283889bac712643eb'), ObjectId('66117a6283889bac712643ec'), ObjectId('66117a6283889bac712643ed'), ObjectId('66117a6283889bac712643ee'), ObjectId('66117a6283889bac712643ef'), ObjectId('66117a6283889bac712643f0'), ObjectId('66117a6283889bac712643f1'), ObjectId('66117a6283889bac712643f2'), ObjectId('66117a6283889bac712643f3'), ObjectId('66117a6283889bac712643f4'), ObjectId('66117a6283889bac712643f5'), ObjectId('66117a6283889bac712643f6'), ObjectId('66117a6283889bac712643f7'), ObjectId('66117a6283889bac712643f8'), ObjectId('66117a6283889bac712643f9'), ObjectId('66117a6283889bac712643fa'), ObjectId('66117a6283889bac712643fb'), ObjectId('66117a6283889bac712643fc'), ObjectId('66117a6283889bac712643fd'), ObjectId('66117a6283889bac712643fe'), ObjectId('66117a6283889bac712643ff'), ObjectId('66117a6283889bac71264400'), ObjectId('66117a6283889bac71264401'), ObjectId('66117a6283889bac71264402'), ObjectId('66117a6283889bac712644

# 6. NLP Pipeline

#### 1. Text cleaning ( Removing ".:;! ...")
#### 2. Tokenization
#### 3. Removing Stop Words
#### 4. Stemming and Lemmatization
#### 5. Part Of Speech (POS)
#### 6. NER

In [2]:
! pip install nltk

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 590.1 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 206.7 kB/s eta 0:00:001m193.0 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 KB 411.5 kB/s eta 0:00:000:00:010:00:01:01


In [84]:
import nltk
from nltk.tokenize import sent_tokenize, WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import ISRIStemmer, WordNetLemmatizer
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

db = client["Lab1"]
articles_collection = db["articles"]

# Get all document
articles = articles_collection.find()

[nltk_data] Downloading package punkt to
[nltk_data]     /home/ayoubbakkali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ayoubbakkali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/ayoubbakkali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ayoubbakkali/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## 6.1 Text cleaning 


## All Arabic Characters

Removing all non arabic caracteres



| Description  | Regular Expression |
| ------------- | ------------- |
| All Arabic characters (1)  | [ء-ي] |
| All Arabic characters (2)  | [\u0600-\u06FF]   |
| Arabic & Persian characters  | [گچپژیلفقهمو ء-ي] |
| Arabic & Persian characters & Hindu–Arabic numbers | [؀-ۿ] |

In [76]:
def text_cleaning(collection, pattern):
    article_cleaning = []
    for article in collection:
        try:
            content_article = article["content"]
            if content_article and len(content_article) > 0:
                # Removing 
                cleaned_content = re.sub(pattern, '', content_article[0])
                article_cleaning.append(cleaned_content)    
        except KeyError:
            # Handle the case where 'content' key is not found in the article
            print("Key 'content' not found in article:", article)
        except Exception as e:
            # Handle other potential errors
            print("Error processing article:", e)
    return article_cleaning 


In [77]:
pattern = r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF\s.,;:!?؟]'
pip1 = text_cleaning(articles,pattern)

In [78]:
pip1

['أخبارعمليات نوعية للقسام توقع  قتيلا من جنود الاحتلال بخان يونس  مدة الفيديو     :آخر تحديث: : م بتوقيت مكة المكرمةكبدت كتائب عز الدين القسام\xa0الجناح العسكري لحركة المقاومة الإسلامية حماس قوات الاحتلال الإسرائيلي  قتيلا وعدة إصابات في عمليات نوعية نفذتها اليوم السبت في محور خان يونس بقطاع غزة.\nوقالت كتائب القسام في بيان إن مقاتليها قتلوا  جنود إسرائيليين، وأصابوا آخرين، في منطقة الزنة شرقي خان يونس، موضحة أنها استهدفت  دبابات ميركافا بقذائف الياسين ، مشيرة إلى أنه فور تقدم قوات الإنقاذ إلى المكان ووصولها إلى وسط حقل ألغام أُعد مسبقا، استُهدفت بتفجير  عبوات مضادة للأفراد.\nوأوضح بيان القسام أن المواجهات لا تزال مستمرة في منطقة الزنة، شرقي خان يونس.\nكما أعلنت كتائب القسام قتل  جنود إسرائيليين من مسافة صفر وإصابة آخرين وتدمير ناقلة جند بمنطقة حي الأمل غرب خان يونس، فضلا عن استهداف دبابة إسرائيلية أخرى بقذيفة الياسين  وقوة راجلة بعبوة وإيقاعها بين قتيل وجريح في خان يونس.\nوأفاد مراسل الجزيرة بهبوط  طائرات مروحية للجيش الإسرائيلي لنقل جرحى من جنود الاحتلال شرقي خان يونس\nمن جهتها، أفا

## 6.2 Tokanization

In [12]:
def sentences_tokenize(articles):
    output = []
    for article in articles:
        output.append(sent_tokenize(article))
    return output

In [13]:
pip2 = sentences_tokenize(pip1)
pip2[0]

['أخبارعمليات نوعية للقسام توقع  قتيلا من جنود الاحتلال بخان يونس  مدة الفيديو     :آخر تحديث: : م بتوقيت مكة المكرمةكبدت كتائب عز الدين القسام\xa0الجناح العسكري لحركة المقاومة الإسلامية حماس قوات الاحتلال الإسرائيلي  قتيلا وعدة إصابات في عمليات نوعية نفذتها اليوم السبت في محور خان يونس بقطاع غزة.',
 'وقالت كتائب القسام في بيان إن مقاتليها قتلوا  جنود إسرائيليين، وأصابوا آخرين، في منطقة الزنة شرقي خان يونس، موضحة أنها استهدفت  دبابات ميركافا بقذائف الياسين ، مشيرة إلى أنه فور تقدم قوات الإنقاذ إلى المكان ووصولها إلى وسط حقل ألغام أُعد مسبقا، استُهدفت بتفجير  عبوات مضادة للأفراد.',
 'وأوضح بيان القسام أن المواجهات لا تزال مستمرة في منطقة الزنة، شرقي خان يونس.',
 'كما أعلنت كتائب القسام قتل  جنود إسرائيليين من مسافة صفر وإصابة آخرين وتدمير ناقلة جند بمنطقة حي الأمل غرب خان يونس، فضلا عن استهداف دبابة إسرائيلية أخرى بقذيفة الياسين  وقوة راجلة بعبوة وإيقاعها بين قتيل وجريح في خان يونس.',
 'وأفاد مراسل الجزيرة بهبوط  طائرات مروحية للجيش الإسرائيلي لنقل جرحى من جنود الاحتلال شرقي خان يونس\nم

## 6.3 Removing Stop words 

In [14]:
print(stopwords.words('arabic'))

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [15]:
# Pre-tokenize stopwords
stop_words = set(stopwords.words('arabic'))
tokenizer = WordPunctTokenizer()

def removing_stopwords(pip):
    sentences_without_stopwords = []
    for article_sentences in pip:
        content_without_stopwords = []
        for sentence in article_sentences:
            words = tokenizer.tokenize(sentence)
            words = [w for w in words if w not in stop_words]
            content_without_stopwords.append(words)
        sentences_without_stopwords.append(content_without_stopwords)
    return sentences_without_stopwords

In [16]:
pip3 = removing_stopwords(pip2)

In [17]:
pip3[0]

[['أخبارعمليات',
  'نوعية',
  'للقسام',
  'توقع',
  'قتيلا',
  'جنود',
  'الاحتلال',
  'بخان',
  'يونس',
  'مدة',
  'الفيديو',
  ':',
  'آخر',
  'تحديث',
  ':',
  ':',
  'بتوقيت',
  'مكة',
  'المكرمةكبدت',
  'كتائب',
  'عز',
  'الدين',
  'القسام',
  'الجناح',
  'العسكري',
  'لحركة',
  'المقاومة',
  'الإسلامية',
  'حماس',
  'قوات',
  'الاحتلال',
  'الإسرائيلي',
  'قتيلا',
  'وعدة',
  'إصابات',
  'عمليات',
  'نوعية',
  'نفذتها',
  'اليوم',
  'السبت',
  'محور',
  'خان',
  'يونس',
  'بقطاع',
  'غزة',
  '.'],
 ['وقالت',
  'كتائب',
  'القسام',
  'بيان',
  'مقاتليها',
  'قتلوا',
  'جنود',
  'إسرائيليين',
  '،',
  'وأصابوا',
  'آخرين',
  '،',
  'منطقة',
  'الزنة',
  'شرقي',
  'خان',
  'يونس',
  '،',
  'موضحة',
  'أنها',
  'استهدفت',
  'دبابات',
  'ميركافا',
  'بقذائف',
  'الياسين',
  '،',
  'مشيرة',
  'أنه',
  'فور',
  'تقدم',
  'قوات',
  'الإنقاذ',
  'المكان',
  'ووصولها',
  'وسط',
  'حقل',
  'ألغام',
  'ُ',
  'عد',
  'مسبقا',
  '،',
  'است',
  'ُ',
  'هدفت',
  'بتفجير',
  'عبوات',
  'مضادة',

## 6.4 Stemming and lemmatization

#### 6.4.1 Stemming

In [18]:
stemmer = ISRIStemmer()

def Stemming_articles(pip):
    stemmed_result = []
    for article_sentences in pip:
        stemmed_article = []
        for sentence in article_sentences:
            stemmed_sentence = [stemmer.stem(word) for word in sentence]
            stemmed_article.append(stemmed_sentence)
        stemmed_result.append(stemmed_article)
    return stemmed_result

In [19]:
pip4_stemming = Stemming_articles(pip3)

In [20]:
pip3[10]

[['أخبارإيران',
  'تعلن',
  'اعتقال',
  'عناصر',
  'بتنظيم',
  'الدولة',
  'خططوا',
  'لهجمات',
  'بعيد',
  'الفطرالشرطة',
  'الإيرانية',
  'قالت',
  'المتهمين',
  'كانوا',
  'يخططون',
  'لهجوم',
  'انتحاري',
  'عيد',
  'الفطر',
  'رويترزأرشيفآخر',
  'تحديث',
  ':',
  ':',
  'بتوقيت',
  'مكة',
  'المكرمةأعلنت',
  'الشرطة',
  'الإيرانية',
  'اليوم',
  'السبت',
  'أنها',
  'ألقت',
  'القبض',
  'عنصر',
  'بارز',
  'تنظيم',
  'الدولة',
  'الإسلامية',
  '،',
  'وعضوين',
  'آخرين',
  'التنظيم',
  'متهمين',
  'بالتخطيط',
  'لهجوم',
  'انتحاري',
  'أثناء',
  'عيد',
  'الفطر',
  '.'],
 ['وذكرت',
  'وسائل',
  'إعلام',
  'إيرانية',
  '،',
  'نقلا',
  'الشرطة',
  '،',
  'محمد',
  'ذاكر',
  'المعروف',
  'باسم',
  'رامش',
  'والعضوين',
  'الآخرين',
  'اعت',
  'ُ',
  'قلوا',
  'كرج',
  'غربي',
  'العاصمة',
  'طهران',
  'أعقاب',
  'اشتباكات',
  '.'],
 ['وقالت',
  'الشرطة',
  'إنها',
  'ألقت',
  'القبض',
  'آخرين',
  'كانوا',
  'بصحبتهم',
  '.'],
 ['وذكرت',
  'وكالة',
  'إرنا',
  'الإيرانية',
  '،',
  

In [21]:
pip4_stemming[10]

[['اخبارإير',
  'تعل',
  'عقل',
  'عنصر',
  'نظم',
  'دول',
  'خطط',
  'هجم',
  'بعد',
  'فطرالشرطة',
  'يرن',
  'قلت',
  'تهم',
  'كان',
  'خطط',
  'هجم',
  'نحر',
  'عيد',
  'فطر',
  'رويترزأرشيفآخر',
  'حدث',
  ':',
  ':',
  'بتق',
  'مكة',
  'مكرمةأعلنت',
  'شرط',
  'يرن',
  'اليوم',
  'سبت',
  'انه',
  'الق',
  'قبض',
  'عنصر',
  'برز',
  'نظم',
  'دول',
  'سلم',
  '،',
  'عضو',
  'اخر',
  'نظم',
  'تهم',
  'خطط',
  'هجم',
  'نحر',
  'ثنء',
  'عيد',
  'فطر',
  '.'],
 ['ذكر',
  'وسل',
  'علم',
  'يرن',
  '،',
  'نقل',
  'شرط',
  '،',
  'حمد',
  'ذكر',
  'عرف',
  'بسم',
  'رمش',
  'عضو',
  'اخر',
  'اعت',
  '',
  'قلا',
  'كرج',
  'غرب',
  'عصم',
  'طهر',
  'عقب',
  'شبك',
  '.'],
 ['وقل', 'شرط', 'انه', 'الق', 'قبض', 'اخر', 'كان', 'صحب', '.'],
 ['ذكر',
  'وكل',
  'ارن',
  'يرن',
  '،',
  'نقل',
  'تحدث',
  'بسم',
  'شرط',
  'عمد',
  'سعد',
  'نظر',
  'هدي',
  '،',
  'سلح',
  'اصب',
  'رصص',
  'طلق',
  'نار',
  'سير',
  '،',
  'وتم',
  'قبض',
  'علي',
  '.'],
 ['أعل',
  'نظم',
  'دول

#### 6.4.2 Lemmatization

In [22]:
! pip install stanza

Defaulting to user installation because normal site-packages is not writeable


In [23]:
import stanza 
stanza.download('ar')  # download Arabic model


/home/ayoubbakkali/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-07 21:09:04 INFO: Downloaded file to /home/ayoubbakkali/stanza_resources/resources.json
2024-04-07 21:09:04 INFO: Downloading default packages for language: ar (Arabic) ...
2024-04-07 21:09:06 INFO: File exists: /home/ayoubbakkali/stanza_resources/ar/default.zip
2024-04-07 21:09:10 INFO: Finished downloading models and saved to /home/ayoubbakkali/stanza_resources


In [24]:
def Lemmatizing_articles(pip):
    nlp = stanza.Pipeline('ar')  # initialize Arabic neural pipeline
    lim_result = []
    for article_sentences in pip:
        stemmed_article = []
        for sentence in article_sentences:
            stemmed_sentence = [nlp(word).sentences[0].words[0].lemma for word in sentence]
            stemmed_article.append(stemmed_sentence)
        lim_result.append(stemmed_article)
    return lim_result

In [25]:
pip4_lemmatization = Lemmatizing_articles(pip3)
pip4_lemmatization[10]

2024-04-07 21:09:17 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-04-07 21:09:17 INFO: Downloaded file to /home/ayoubbakkali/stanza_resources/resources.json
2024-04-07 21:09:18 INFO: Loading these models for language: ar (Arabic):
| Processor | Package       |
-----------------------------
| tokenize  | padt          |
| mwt       | padt          |
| pos       | padt_charlm   |
| lemma     | padt_nocharlm |
| depparse  | padt_charlm   |
| ner       | aqmar_charlm  |

2024-04-07 21:09:18 INFO: Using device: cpu
2024-04-07 21:09:18 INFO: Loading: tokenize
2024-04-07 21:09:19 INFO: Loading: mwt
2024-04-07 21:09:19 INFO: Loading: pos
2024-04-07 21:09:19 INFO: Loading: lemma
2024-04-07 21:09:19 INFO: Loading: depparse
2024-04-07 21:09:19 INFO: Loading: ner
2024-04-07 21:09:20 INFO: Done loading processors!


[['أخبارإيران',
  'أَعلَن',
  'اِعتِقَال',
  'عُنصُر',
  'بُتَنظِيم',
  'دَولَة',
  'خَطَّط',
  'لَهجَمَات',
  'بُعَيدَ',
  'فَطرَالشرَة',
  'إِيرَانِيّ',
  'قَال',
  'مُتَّهَم',
  'كَان',
  'خَطَّط',
  'لَهجُوم',
  'اِنتِحَارِيّ',
  'عيد',
  'فِطر',
  'رُويترزأرشِيفِآخُر',
  'تَحدِيث',
  ':',
  ':',
  'بُتُوقِيت',
  'مَكَّة',
  'مُكَرَّم',
  'شُرطَة',
  'إِيرَانِيّ',
  'يَوم',
  'سَبت',
  'أَنَّ',
  'أَلقَى',
  'قَبض',
  'عُنصُر',
  'بَارِز',
  'تَنظِيم',
  'دَولَة',
  'إِسلَامِيّ',
  '،',
  'وعضوين',
  'آخَر',
  'تَنظِيم',
  'مُتَّهِم',
  'تَخطِيط',
  'لَهجُوم',
  'اِنتِحَارِيّ',
  'أَثنَاءَ',
  'عيد',
  'فِطر',
  '.'],
 ['وَ',
  'وَ',
  'إِعلَام',
  'إِيرَانِيّ',
  '،',
  'نَقل',
  'شُرطَة',
  '،',
  'محمد',
  'ذاكر',
  'مَعرُوف',
  'بَاسِم',
  'رامش',
  'عُضو',
  'آخَر',
  'أَعَتّ',
  'ُ',
  'قَلَا',
  'كرج',
  'غَربِيّ',
  'عَاصِمَة',
  'طَهرَان',
  'عُقب',
  'اِشتِبَاك',
  '.'],
 ['وَ', 'شُرطَة', 'إِنَّ', 'أَلقَى', 'قَبض', 'آخَر', 'كَان', 'هُوَ', '.'],
 ['وَ',
  'وِكَالَة',
  'إر

In [41]:
pip3[10]

[['أخبارإيران',
  'تعلن',
  'اعتقال',
  'عناصر',
  'بتنظيم',
  'الدولة',
  'خططوا',
  'لهجمات',
  'بعيد',
  'الفطرالشرطة',
  'الإيرانية',
  'قالت',
  'المتهمين',
  'كانوا',
  'يخططون',
  'لهجوم',
  'انتحاري',
  'عيد',
  'الفطر',
  'رويترزأرشيفآخر',
  'تحديث',
  ':',
  ':',
  'بتوقيت',
  'مكة',
  'المكرمةأعلنت',
  'الشرطة',
  'الإيرانية',
  'اليوم',
  'السبت',
  'أنها',
  'ألقت',
  'القبض',
  'عنصر',
  'بارز',
  'تنظيم',
  'الدولة',
  'الإسلامية',
  '،',
  'وعضوين',
  'آخرين',
  'التنظيم',
  'متهمين',
  'بالتخطيط',
  'لهجوم',
  'انتحاري',
  'أثناء',
  'عيد',
  'الفطر',
  '.'],
 ['وذكرت',
  'وسائل',
  'إعلام',
  'إيرانية',
  '،',
  'نقلا',
  'الشرطة',
  '،',
  'محمد',
  'ذاكر',
  'المعروف',
  'باسم',
  'رامش',
  'والعضوين',
  'الآخرين',
  'اعت',
  'ُ',
  'قلوا',
  'كرج',
  'غربي',
  'العاصمة',
  'طهران',
  'أعقاب',
  'اشتباكات',
  '.'],
 ['وقالت',
  'الشرطة',
  'إنها',
  'ألقت',
  'القبض',
  'آخرين',
  'كانوا',
  'بصحبتهم',
  '.'],
 ['وذكرت',
  'وكالة',
  'إرنا',
  'الإيرانية',
  '،',
  

Stemming and lemmatization are two common techniques used in natural language processing to reduce words to their root forms, but they operate differently. Stemming involves removing affixes and suffixes from words to obtain their base or root form. However, stemming may not always produce accurate results, especially for languages with complex morphology like Arabic.

For instance, in the example above (pip3[0]), the word "الإيرانية" (meaning "Iranian" in English) stemmed to "يرن" by ISRIStemmer of nltk package, which doesn't fully capture its original meaning or form.

On the other hand, lemmatization, as implemented by libraries like Stanza, offers a more sophisticated approach. Stanza employs neural networks to analyze words in context and determine their lemmas, or dictionary forms. This process considers factors such as part of speech, context, and morphology to accurately identify the base form of words.

In the case of the Arabic word "الإيرانية", Stanza's lemmatization would correctly identify its lemma as "إيراني", preserving its semantic meaning and morphology.

### 6.5 POS

#### 6.5.1 POS : Machine Learning Approche

In [29]:
def POS_articles(pip):
    nlp = stanza.Pipeline('ar')  # initialize Arabic neural pipeline
    lim_result = []
    for article_sentences in pip:
        tagged_article = []
        for sentence in article_sentences:
            tagged_sentence = []
            doc = nlp(" ".join(sentence))
            for word in doc.sentences[0].words:
                tagged_word = (word.text, word.upos)  # Tuple of word and POS tag
                tagged_sentence.append(tagged_word)
            tagged_article.append(tagged_sentence)
        lim_result.append(tagged_article)
    return lim_result


In [30]:
pip5 = POS_articles(pip4_lemmatization)

2024-04-07 21:51:18 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-04-07 21:51:18 INFO: Downloaded file to /home/ayoubbakkali/stanza_resources/resources.json
2024-04-07 21:51:19 INFO: Loading these models for language: ar (Arabic):
| Processor | Package       |
-----------------------------
| tokenize  | padt          |
| mwt       | padt          |
| pos       | padt_charlm   |
| lemma     | padt_nocharlm |
| depparse  | padt_charlm   |
| ner       | aqmar_charlm  |

2024-04-07 21:51:19 INFO: Using device: cpu
2024-04-07 21:51:19 INFO: Loading: tokenize
2024-04-07 21:51:19 INFO: Loading: mwt
2024-04-07 21:51:19 INFO: Loading: pos
2024-04-07 21:51:19 INFO: Loading: lemma
2024-04-07 21:51:20 INFO: Loading: depparse
2024-04-07 21:51:20 INFO: Loading: ner
2024-04-07 21:51:20 INFO: Done loading processors!


In [79]:
pip5[0]

[[('أَخبَارعْمِيَّة', 'X'),
  ('نَوعِيّ', 'X'),
  ('قَاسِم', 'X'),
  ('تَوَقَّع', 'VERB'),
  ('قَتِيل', 'X'),
  ('جُندِيّ', 'X'),
  ('اِحتِلَال', 'X'),
  ('ب', 'ADP'),
  ('خان', 'X'),
  ('يُونِس', 'X'),
  ('مُدَّة', 'X'),
  ('الفيديو', 'X'),
  (':', 'PUNCT'),
  ('آخَر', 'ADJ'),
  ('تَحدِيث', 'X'),
  (':', 'PUNCT'),
  (':', 'PUNCT'),
  ('ب', 'ADP'),
  ('توق<UNK>يت', 'NOUN'),
  ('مَكَّة', 'X'),
  ('مُكَرَّم', 'X'),
  ('كَتِيبَة', 'X'),
  ('عز', 'X'),
  ('دِين', 'X'),
  ('القسام', 'X'),
  ('جَنَاح', 'X'),
  ('عَسكَرِيّ', 'X'),
  ('ل', 'ADP'),
  ('َحرَكَة', 'NOUN'),
  ('مُقَاوَمَة', 'X'),
  ('إِسلَامِيّ', 'X'),
  ('حماس', 'X'),
  ('قُوَّة', 'X'),
  ('اِحتِلَال', 'X'),
  ('إِسرَائِيلِيّ', 'X'),
  ('قَتِيل', 'X'),
  ('و', 'CCONJ'),
  ('َ', 'VERB'),
  ('عدَة', 'X'),
  ('إِصَابَة', 'X'),
  ('عَمَلِيَّة', 'X'),
  ('نَوعِيّ', 'X'),
  ('نَفَّذ', 'VERB'),
  ('يَوم', 'X'),
  ('سَبت', 'X'),
  ('محور', 'X'),
  ('خان', 'X'),
  ('يُونِس', 'X'),
  ('ب', 'ADP'),
  ('َقطَاع', 'NOUN'),
  ('غَزَّة', 'X'),
 

#### 6.5.2 POS : Rule Based

In [91]:
def POS_articles_nltk(pip):
    result = []
    for article_sentences in pip:
        pos_article = []
        for sentence in article_sentences:
            pos_article.append(nltk.pos_tag(sentence))
        result.append(pos_article)
    return result

In [92]:
pip5_rulebased = POS_articles_nltk(pip4_lemmatization)

In [94]:
pip5_rulebased[10]

[[('أخبارإيران', 'JJ'),
  ('أَعلَن', 'NNP'),
  ('اِعتِقَال', 'NNP'),
  ('عُنصُر', 'NNP'),
  ('بُتَنظِيم', 'NNP'),
  ('دَولَة', 'NNP'),
  ('خَطَّط', 'NNP'),
  ('لَهجَمَات', 'NNP'),
  ('بُعَيدَ', 'NNP'),
  ('فَطرَالشرَة', 'NNP'),
  ('إِيرَانِيّ', 'NNP'),
  ('قَال', 'NNP'),
  ('مُتَّهَم', 'NNP'),
  ('كَان', 'NNP'),
  ('خَطَّط', 'NNP'),
  ('لَهجُوم', 'NNP'),
  ('اِنتِحَارِيّ', 'NNP'),
  ('عيد', 'NNP'),
  ('فِطر', 'NNP'),
  ('رُويترزأرشِيفِآخُر', 'NNP'),
  ('تَحدِيث', 'NN'),
  (':', ':'),
  (':', ':'),
  ('بُتُوقِيت', 'NN'),
  ('مَكَّة', 'NNP'),
  ('مُكَرَّم', 'NNP'),
  ('شُرطَة', 'NNP'),
  ('إِيرَانِيّ', 'NNP'),
  ('يَوم', 'NNP'),
  ('سَبت', 'NNP'),
  ('أَنَّ', 'NNP'),
  ('أَلقَى', 'NNP'),
  ('قَبض', 'NNP'),
  ('عُنصُر', 'NNP'),
  ('بَارِز', 'NNP'),
  ('تَنظِيم', 'NNP'),
  ('دَولَة', 'NNP'),
  ('إِسلَامِيّ', 'NNP'),
  ('،', 'NNP'),
  ('وعضوين', 'NNP'),
  ('آخَر', 'NNP'),
  ('تَنظِيم', 'NNP'),
  ('مُتَّهِم', 'NNP'),
  ('تَخطِيط', 'NNP'),
  ('لَهجُوم', 'NNP'),
  ('اِنتِحَارِيّ', 'NNP'),
  ('

### 6.6 NER tag

In [115]:
def NER_articles_stanza(pip):
    # Initialize Stanza pipeline for Arabic with NER enabled
    nlp = stanza.Pipeline('ar', processors='tokenize, ner')
    
    lim_result = []
    for article_sentences in pip:
        tagged_article = []
        for sentence in article_sentences:
            # Process the sentence to perform NER tagging
            doc = nlp(" ".join(sentence))
            ner_sentence = [(ent.text, ent.type) for sent in doc.sentences for ent in sent.ents]
            tagged_article.append(ner_sentence)
        lim_result.append(tagged_article)
    return lim_result

In [116]:
pip_6 = NER_articles_stanza(pip3)

2024-04-08 01:55:43 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-04-08 01:55:44 INFO: Downloaded file to /home/ayoubbakkali/stanza_resources/resources.json
2024-04-08 01:55:44 WARNING: Language ar package default expects mwt, which has been added
2024-04-08 01:55:44 INFO: Loading these models for language: ar (Arabic):
| Processor | Package      |
----------------------------
| tokenize  | padt         |
| mwt       | padt         |
| ner       | aqmar_charlm |

2024-04-08 01:55:44 INFO: Using device: cpu
2024-04-08 01:55:44 INFO: Loading: tokenize
2024-04-08 01:55:44 INFO: Loading: mwt
2024-04-08 01:55:44 INFO: Loading: ner
2024-04-08 01:55:45 INFO: Done loading processors!


In [120]:
pip_6[2]

[[('لبنان', 'LOC'), ('لبنان', 'LOC'), ('لبنان', 'LOC')],
 [('غارتين', 'PER'), ('عيتا', 'LOC'), ('أرنون', 'LOC'), ('لبنان', 'LOC')],
 [('وكالة الأنباء اللبنانية', 'ORG'),
  ('الغارات الإسرائيلية', 'ORG'),
  ('عيتا', 'LOC')],
 [('تلة حمامص', 'LOC'), ('لبنان', 'LOC')],
 [('الخيام', 'LOC')],
 [('جنوب لبنان', 'LOC'),
  ('لبنان', 'LOC'),
  ('الجيش الإسرائيلي', 'ORG'),
  ('دو', 'PER'),
  ('الحدود اللبنانية', 'LOC')],
 [('إسرائيليين', 'PER')],
 [('حزب الله', 'ORG'),
  ('الجيش الإسرائيلي', 'ORG'),
  ('لبنان', 'LOC'),
  ('أكتوبرتشرين الأول', 'PER')],
 [('وميس الجبل', 'PER')],
 [('للبنان', 'LOC')],
 [('غزة', 'LOC'), ('أكتوبرتشرين', 'LOC'), ('الجيش الإسرائيلي', 'ORG')],
 []]